## Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras 
import tensorflow as tf
from tensorflow.keras import layers
import keras.backend as K
import ipywidgets as widgets
from IPython.display import display


## Load Data

In [ ]:
data = pd.read_csv('../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')
data.head()

## Prepare data

In [ ]:
data = data.pixels.apply(lambda x: np.array(x.split(" "),dtype = float))
arr = np.stack(data)
arr = arr / 255.0
arr = arr.astype('float32')
arr = arr.reshape(arr.shape[0],48,48,1)


## Visualize our data


In [ ]:
plt.figure(figsize = (10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(arr[i],cmap = 'gray')
    plt.axis('off')

## Defining Batch Size and shuffling data

In [ ]:
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices(arr).batch(batch_size).shuffle(132)     


## Sampling 

In [ ]:
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        z_mean , z_log_var = inputs;
        batch = tf.shape(z_mean)[0]
        dimension = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape = (batch, dimension))
        return z_mean + tf.exp(z_log_var * 0.5) * epsilon

## Defining our encoder model

In [ ]:
latent_dim = 3
encoder_inputs = layers.Input(shape = (48,48,1))
x = layers.Conv2D(filters = 32, kernel_size = (2,2), activation = 'relu', padding = 'same')(encoder_inputs)
x = layers.Conv2D(filters = 64, kernel_size = (2,2), activation = 'relu')(x)
x = layers.MaxPool2D(pool_size = (2,2))(x)
x = layers.Conv2D(filters = 64, kernel_size = (2,2), activation = 'relu')(x)
x = layers.Conv2D(filters = 128, kernel_size = (2,2), activation = 'relu')(x)
x = layers.MaxPool2D(pool_size = (2,2))(x)
x = layers.Conv2D(filters = 128, kernel_size = (2,2), activation = 'relu')(x)
x = layers.Conv2D(filters = 256, kernel_size = (2,2), activation = 'relu')(x)
x = layers.MaxPool2D(pool_size = (2,2))(x)

x = layers.Flatten()(x)
mean = layers.Dense(latent_dim)(x)
log_var = layers.Dense(latent_dim)(x)
z = Sampling()([mean, log_var])

encoder = tf.keras.Model(encoder_inputs, [mean,log_var,z])

encoder.summary()


## Defining Decoder Model

In [ ]:
decoder_input = layers.Input(shape = (latent_dim,))

x = layers.Dense(6*6*32, activation = 'relu')(decoder_input)

x = layers.Reshape((6,6,32))(x)
x = layers.Conv2D(filters = 64, kernel_size = (2,2), padding = 'same',activation = 'relu')(x)
x = layers.Conv2D(filters = 64, kernel_size = (2,2), padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.UpSampling2D()(x)
x = layers.Conv2D(filters = 64, kernel_size = (2,2), padding = 'same',activation = 'relu')(x)
x = layers.Conv2D(filters = 128, kernel_size = (2,2), padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.UpSampling2D()(x)
x = layers.Conv2D(filters = 128, kernel_size = (2,2), padding = 'same',activation = 'relu')(x)
x = layers.Conv2D(filters = 256, kernel_size = (2,2), padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters = 256, kernel_size = (2,2), padding = 'same',activation = 'relu')(x)
x = layers.Conv2D(filters =128, kernel_size = (2,2), padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.UpSampling2D()(x)
x = layers.Conv2D(filters = 1, kernel_size = (2,2), padding = 'same')(x)

decoder = tf.keras.Model(decoder_input, x)
decoder.summary()
 

## VAE Model

In [ ]:
inp = encoder.input
out = encoder.output
decoder_output = decoder(out[2])
vae = tf.keras.Model(inp, decoder_output)
vae.summary()

## Defining Loss functions

In [ ]:
def kl_loss(z_log_var,z_mean):
    kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
    return kl_loss * 0.012


loss = tf.keras.losses.BinaryCrossentropy()
def reconstruction_loss(data,reconstructed):
    return loss(data,reconstructed)
optimizer = tf.keras.optimizers.Adam()    

## Train steps

In [ ]:
def train_steps(data):
    with tf.GradientTape() as vae_tape:
        z_mean,z_log_var,z = encoder(data)
        reconstructed_image = decoder(z)
        kl_ = kl_loss(z_log_var,z_mean)
        reconstruction_ = reconstruction_loss(data, reconstructed_image)
        total_loss = kl_ + reconstruction_

    gradient2 = vae_tape.gradient(total_loss, vae.trainable_variables)
    optimizer.apply_gradients(zip(gradient2, vae.trainable_variables))

In [ ]:
noise = tf.keras.backend.random_normal(shape = (9,latent_dim))
def visualize(epoch):
        prediction = decoder(noise)
        plt.figure(figsize = (5,5))
        for i in range(9):
            if(i == 2):
                plt.title("Epoch: {}".format(epoch))
            plt.subplot(3,3,i+1)
            plt.imshow(prediction[i],cmap = 'gray')
            plt.axis('off')

In [ ]:
import time
import numpy as np
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        for data in dataset:
            train_steps(data)
        print("Epoch: {} Time: {}".format(epoch+1,np.round(time.time()-start),3))
        if epoch % 3 == 0:
            visualize(epoch+1)

## Training

In [ ]:
train(dataset,15)

## To use widgets for interactive visualization
In order to use this widgets you have to copy and run this model

In [ ]:
def generate_image(latent1, latent2, latent3):
    latent_values = np.array([[latent1, latent2, latent3]])
    reconstruction = np.array(decoder(latent_values))
    reconstruction = reconstruction.reshape(48,48,1)
    plt.figure(figsize = (4,4))
    plt.imshow(reconstruction,cmap = 'gray')
    plt.axis('off')
    plt.show()

In [ ]:
import numpy as np
a,b,z = encoder(np.array(arr[:5000]).reshape(np.array(arr[:5000]).shape[0],48,48,1))
latent1_min = np.min(z[:,0])-1
latent1_max = np.max(z[:,0])+1

latent2_min = np.min(z[:,1])-1
latent2_max = np.max(z[:,1])+1

latent3_min =np.min(z[:,2])-1
latent3_max = np.max(z[:,2])+1


In [ ]:

face_image_generator = widgets.interact(
    generate_image,
    latent1=(latent1_min, latent1_max),
    latent2=(latent2_min, latent2_max),
    latent3=(latent3_min, latent3_max),
)

display(generate_image)
##  copy and run this notebook to use this widget

In [ ]:
# visualize our input image through VAE
i = np.random.randint(1,2323)
out = vae.predict(arr[i].reshape(1,48,48,1))
plt.subplot(1,2,1)
plt.title("Original Image")
plt.imshow(arr[i], cmap = 'gray')
plt.subplot(1,2,2)
plt.title("Reconstructed Image")
plt.imshow(out.reshape(48,48,1), cmap = 'gray')

## Thank You 
## Any Suggestion to improve this notebook is highly appreciated

ref: <a href= 'https://www.kaggle.com/gcdatkin/an-introduction-to-variational-autoencoders'> here </a>